<a href="https://colab.research.google.com/github/rakehsaleem/Crack-Detection-TF-1.x.x/blob/master/Mask_R_CNN_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mask R-CNN instance segmentation with custom dataset in Google Colab**

Jupyter notebook providing steps to train a **Mask R-CNN** model with custom dataset. Requirements are only dataset images and annotations file which you can get it from [here](https://github.com/rakehsaleem/Crack-Detection-TF-1.x.x/releases/tag/v1.0)

**Colab Runtime type: Python3, GPU enabled.**

#**Making Dataset**

I generated dataset annotations with [VGG Image Annotator](https://www.robots.ox.ac.uk/~vgg/software/via/).

Notebook train a model for one class object detection. It is possible to slightly modify notebook to train model for multiple classes.

Before running notebook, we need to create dataset:

1. Collect various pictures of objects to detect
2. Create annotation files in VGG
3. Create image.zip file having structure defined below
4. Upload the zip file in your Google Drive

Zip file structure:
```
images.zip
|- "train" directory
  |- jpg image files of training data
  |- "via_region_data.json" annotations file of training data
|- "val" directory
  |- jpg image files of validation data
  |- "via_region_data.json" annotations file of validation data
```

Check my image.zip file as dataset example.


# Install required packages

In [2]:
%cd

!git clone --quiet https://github.com/rakehsaleem/Crack-Detection-TF-1.x.x.git

/root


In [3]:
%cd ~/Crack-Detection-TF-1.x.x

!pip uninstall -y tensorflow
!pip install tensorflow==1.14.0
!pip install keras==2.2.4
import tensorflow as tf
print(tf.__version__)
!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

/root/Crack-Detection-TF-1.x.x
Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
     |████████████████████████████████| 109.3 MB 46 kB/s 
     |████████████████████████████████| 488 kB 68.4 MB/s 
     |████████████████████████████████| 3.1 MB 28.0 MB/s 
     |████████████████████████████████| 50 kB 8.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Uninstalling tensorboard-2.6.0:
      Successfully uninstalled tensorboard-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.5 requires tensorflow>=2.0.0, but you have tensorflow

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0
     |████████████████████████████████| 63 kB 1.9 MB/s 
ERROR: Could not find a version that satisfies the requirement m2w64-toolchain (from versions: none)
ERROR: No matching distribution found for m2w64-toolchain
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'description-file' will not be supported in future versions. Please use the underscore name 'description_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'license-file' will not be supported in future versions. Please use the underscore name 'license_file' instead
  % (opt, underscore_opt))
/usr/local/lib/python3.7/dist-packages/setuptools/dist.py:700: UserWarning: Usage of dash-separated 'requirements-file' will not be supported in future versions. Please use the underscore name 'requirements_file' instead
  % (opt, underscore_opt))
running install
running bdist_egg
running egg_

**Dataset Preparation and set up locations**

In [4]:
%cd ~/Crack-Detection-TF-1.x.x
fileId = '12bCRgVoRaegFY7xQ2h6qmFrMKz8AtLZt'

import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

os.makedirs('dataset')
os.chdir('dataset')

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fileName = fileId + '.zip'
downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile(fileName)
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)

/root/Crack-Detection-TF-1.x.x
Extracted zip file 12bCRgVoRaegFY7xQ2h6qmFrMKz8AtLZt.zip


In [5]:
%cd ~/Crack-Detection-TF-1.x.x

!cp ~/Crack-Detection-TF-1.x.x/crack/crack.py ./crack.py

!sed -i -- 's/epochs=30/epochs=5/g' crack.py

/root/Crack-Detection-TF-1.x.x


# Training Starts
If for some reason you get error related to keras, locate the file with error and go to file line to modify these two lines:

```
original_keras_version = f.attrs['keras_version'].decode('utf8')
original_backend = f.attrs['backend'].decode('utf8')

to

original_keras_version = f.attrs['keras_version']
original_backend = f.attrs['backend']
```

In [ ]:
%cd ~/Crack-Detection-TF-1.x.x

!python crack.py train --dataset=dataset/Dataset3 --weights=coco

/root/Crack-Detection-TF-1.x.x
Using TensorFlow backend.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/fram